# Summarize Assignment of On-board Survey Demand
This notebook uses the Acceptance Criteria infrastructure to summarize the results of the assignment of the on-board survey demand to the TM2 networks. 

In [1]:
from tm2py.acceptance.canonical import Canonical
from tm2py.acceptance.simulated import Simulated
from tm2py.acceptance.observed import Observed
from tm2py.acceptance.acceptance import Acceptance

import pandas as pd

Inro not found. Skipping inro setup.


### Remote I/O

In [2]:
scenario_config_file = "../examples/scenario_config.toml"
observed_config_file = "../examples/observed_data.toml"
canonical_crosswalk_file = "../examples/canonical_crosswalk.toml"

input_previous_summary_filename = "on-board-assignment-summary.csv"

output_folder_root = "/Users/wsp/Documents/GitHub/tm2py/examples/temp_on_board_assign/"
output_filename = "on-board-assignment-summary.csv"

## Parameters

In [3]:
append_or_create = "create" 
iteration_description = "test"
iteration_number = 0

In [4]:
c = Canonical(canonical_crosswalk_file, on_board_assign_summary=True)

In [5]:
s = Simulated(canonical=c, scenario_file=scenario_config_file, model_file=None, on_board_assign_summary=True)
s.reduce_on_board_assignment_boardings(time_period_list=["am"])

In [6]:
s.simulated_boardings_df.head()

daily_line_name  tm2_mode line_mode        operator technology  fare_system  \
0      10_573_198        12         o  Emery Go-Round  Local bus           18   
1      10_573_199        12         o  Emery Go-Round  Local bus           18   
2      10_573_200        12         o  Emery Go-Round  Local bus           18   
3      10_573_201        12         o  Emery Go-Round  Local bus           18   
4      10_573_202        12         o  Emery Go-Round  Local bus           18   

   total_boarding  total_hour_cap time_period            line_name  
0             0.0       33.750000       daily  N/A -- Daily Record  
1             0.0      207.692308       daily  N/A -- Daily Record  
2             0.0      207.692308       daily  N/A -- Daily Record  
3             0.0       33.750000       daily  N/A -- Daily Record  
4             0.0      192.857143       daily  N/A -- Daily Record

In [7]:
o = Observed(canonical=c, observed_file=observed_config_file, on_board_assign_summary=True)
o.reduce_on_board_survey()

In [8]:
a = Acceptance(canonical=c, simulated=s, observed=o, output_file_root=output_folder_root)  
compare_df = a.compare_transit_boardings()

In [9]:
compare_df.head()

tm2_mode line_mode  operator     technology time_period  total_boarding  \
0      14.0         o  Caltrain      Local Bus          am        0.000000   
1      14.0         o  Caltrain      Local Bus       daily        0.000000   
2     120.0         h      BART     Heavy Rail          am     2692.072717   
3     120.0         h      BART     Heavy Rail       daily     2692.072717   
4     130.0         i  Caltrain  Commuter Rail          am        0.000000   

  survey_route  survey_boardings  florida_threshold daily_line_name  ...  \
0          NaN               NaN                NaN             NaN  ...   
1          NaN               NaN                NaN             NaN  ...   
2         BART     134161.945802                NaN             NaN  ...   
3         BART     431650.949580               0.25             NaN  ...   
4     Caltrain      20978.587956                NaN             NaN  ...   

   line_name  survey_tech survey_operator survey_agency standard_route_id  \
0        NaN          NaN             NaN           NaN               NaN   
1        NaN          NaN             NaN           NaN               NaN   
2        NaN          NaN             NaN           NaN               NaN   
3        NaN          NaN             NaN           NaN               NaN   
4        NaN          NaN             NaN           NaN               NaN   

  canonical_operator standard_route_short_name standard_route_long_name  \
0                NaN                       NaN                      NaN   
1                NaN                       NaN                      NaN   
2                NaN                       NaN                      NaN   
3                NaN                       NaN                      NaN   
4                NaN                       NaN                      NaN   

  standard_line_name standard_headsign  
0                NaN               NaN  
1                NaN               NaN  
2                NaN               NaN  
3                NaN               NaN  
4                NaN               NaN  

[5 rows x 22 columns]

## Make Output

In [10]:
compare_df["iteration_description"] = iteration_description
compare_df["iteration_number"] = iteration_number

if append_or_create == "create":
    compare_df.to_csv(output_folder_root + output_filename, index=False)
else:
    previous_df = pd.read_csv(output_folder_root + input_previous_summary_filename, dtype=compare_df.dtypes.to_dict())
    pd.concat([previous_df, compare_df], ignore_index=True).to_csv(output_folder_root + output_filename, index=False)